In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls
!pwd
!cd /content/drive
print('done')

Mounted at /content/drive
drive  sample_data
/content
done


In [ ]:
import pandas as pd
import pickle
import numpy as np
import networkx as nx
from itertools import islice
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import xml.etree.ElementTree as ET
import re

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/graphstuff/chem_gene_onehotencoded.csv")
G = nx.read_gexf("/content/drive/MyDrive/graphstuff/graph.gexf")

In [ ]:
file_path = '/content/drive/MyDrive/graphstuff/100000/sage_embeddings_mainmap.pkl' #maps chemical or gene ids to index and embedding array

with open(file_path, 'rb') as file:
    embeddings_mainmap = pickle.load(file)

print('\n'.join(islice(embeddings_mainmap, 10)))
print('\n')
print(len(embeddings_mainmap))
print('\n')
embeddings_mainmap['C534883']

C534883
367.0
4609.0
C112297
3784.0
3785.0
C039775
1557.0
C534422
836.0


23189




[[0],
 array([ 0.07876712, -0.05969036,  0.08527903,  0.01284475,  0.01780216,
         0.01634571, -0.09160165,  0.16863525, -0.0280142 , -0.10524438,
         0.0598112 , -0.06157446,  0.05455557, -0.02184562,  0.0820353 ,
        -0.03676902,  0.09297118,  0.01067702, -0.13881788, -0.14166732,
         0.0510341 , -0.12559067, -0.04321984, -0.00853302, -0.06097117,
        -0.02221368, -0.0600847 , -0.22527489, -0.191195  , -0.04995764,
         0.10439263, -0.13660212, -0.07961906,  0.05463179, -0.03425846,
         0.23327908,  0.01161615,  0.01123109,  0.10467409,  0.0134046 ,
         0.01144198,  0.04747581, -0.16887026, -0.22795793,  0.14039843,
        -0.3147707 ,  0.03647125, -0.05182172,  0.02437897, -0.05491275,
         0.13597223,  0.05589329,  0.24718064, -0.15595333, -0.17025773,
        -0.13367224, -0.09391661, -0.25594395, -0.1517817 ,  0.14614582,
        -0.13768502, -0.00587407, -0.2343698 ,  0.03581745], dtype=float32)]

In [ ]:
file_path = '/content/drive/MyDrive/graphstuff/100000/sage_embeddings_reversemap.pkl' #maps index to chemical or gene ids and embedding array

with open(file_path, 'rb') as file:
    embeddings_reversemap = pickle.load(file)
print(len(embeddings_reversemap))
print('\n')
embeddings_reversemap[0]

23189




[['C534883'],
 array([ 0.07876712, -0.05969036,  0.08527903,  0.01284475,  0.01780216,
         0.01634571, -0.09160165,  0.16863525, -0.0280142 , -0.10524438,
         0.0598112 , -0.06157446,  0.05455557, -0.02184562,  0.0820353 ,
        -0.03676902,  0.09297118,  0.01067702, -0.13881788, -0.14166732,
         0.0510341 , -0.12559067, -0.04321984, -0.00853302, -0.06097117,
        -0.02221368, -0.0600847 , -0.22527489, -0.191195  , -0.04995764,
         0.10439263, -0.13660212, -0.07961906,  0.05463179, -0.03425846,
         0.23327908,  0.01161615,  0.01123109,  0.10467409,  0.0134046 ,
         0.01144198,  0.04747581, -0.16887026, -0.22795793,  0.14039843,
        -0.3147707 ,  0.03647125, -0.05182172,  0.02437897, -0.05491275,
         0.13597223,  0.05589329,  0.24718064, -0.15595333, -0.17025773,
        -0.13367224, -0.09391661, -0.25594395, -0.1517817 ,  0.14614582,
        -0.13768502, -0.00587407, -0.2343698 ,  0.03581745], dtype=float32)]

In [ ]:
X = np.load("/content/drive/MyDrive/graphstuff/100000/X.npy") # embeddings
y = np.load("/content/drive/MyDrive/graphstuff/100000/y.npy") #

print(len(X))
print(len(y))
print(X[:1])
print(y[:1])

192647
192647
[[ 7.87671208e-02 -5.96903563e-02  8.52790326e-02  1.28447488e-02
   1.78021640e-02  1.63457058e-02 -9.16016474e-02  1.68635249e-01
  -2.80141979e-02 -1.05244376e-01  5.98112009e-02 -6.15744591e-02
   5.45555726e-02 -2.18456164e-02  8.20352957e-02 -3.67690213e-02
   9.29711759e-02  1.06770154e-02 -1.38817877e-01 -1.41667321e-01
   5.10341041e-02 -1.25590667e-01 -4.32198383e-02 -8.53302144e-03
  -6.09711707e-02 -2.22136788e-02 -6.00847006e-02 -2.25274891e-01
  -1.91194996e-01 -4.99576405e-02  1.04392633e-01 -1.36602119e-01
  -7.96190575e-02  5.46317883e-02 -3.42584550e-02  2.33279079e-01
   1.16161462e-02  1.12310871e-02  1.04674086e-01  1.34046003e-02
   1.14419796e-02  4.74758148e-02 -1.68870255e-01 -2.27957934e-01
   1.40398428e-01 -3.14770699e-01  3.64712477e-02 -5.18217236e-02
   2.43789740e-02 -5.49127497e-02  1.35972232e-01  5.58932945e-02
   2.47180641e-01 -1.55953333e-01 -1.70257732e-01 -1.33672237e-01
  -9.39166099e-02 -2.55943954e-01 -1.51781693e-01  1.46145821e

In [ ]:


random_seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_seed)

clf = MLPClassifier(hidden_layer_sizes=[64, 32, 16], alpha=1e-3, batch_size=4, solver='adam', early_stopping=True, random_state=random_seed)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6566380133715377

In [ ]:


random_seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

ros = RandomOverSampler(sampling_strategy='all', random_state=random_seed)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Create and train the MLPClassifier
clf_resampled = MLPClassifier(hidden_layer_sizes=[64, 32, 16], alpha=1e-3, batch_size=4, solver='adam', early_stopping=True, random_state=random_seed)
clf_resampled.fit(X_train_resampled, y_train_resampled)

# Calculate and print the accuracy score on the test set
accuracy = clf_resampled.score(X_test, y_test)
print("Accuracy: {:.2f}".format(accuracy))


Accuracy: 0.09


In [ ]:
#adjust weight of classifer

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = clf.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average='micro')

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 Score: {:.2f}".format(f1))

Accuracy: 0.66
Precision: 0.58
Recall: 0.32
F1 Score: 0.41


In [ ]:
y_pred = clf_resampled.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average='micro')

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 Score: {:.2f}".format(f1))

In [ ]:


# Specify the path to your XML file
xml_file_path = "/content/drive/MyDrive/graphstuff/test_PMC10518935.xml"

# Read the XML content from the file
with open(xml_file_path, "r", encoding="utf-8") as file:
    xml_text = file.read()

root = ET.fromstring(xml_text)

# Iterate through passages
for passage in root.findall(".//passage"):
    passage_text = passage.find(".//text").text

    # Extract annotations from the passage
    annotations = passage.findall(".//annotation")

    # Create sets to store chemicals and genes
    chemicals_set = set()
    genes_set = set()

    # Iterate through annotations in the passage
    for annotation in annotations:
        annotation_type = annotation.find(".//infon[@key='type']").text
        annotation_id_element = annotation.find(".//infon[@key='identifier']")
        annotation_id = annotation_id_element.text if annotation_id_element is not None else None
        annotation_text = annotation.find(".//text").text

        # Check if the annotation is a Chemical or Gene and add it to the respective set
        if annotation_type == 'Chemical':
            chemicals_set.add((annotation_text, annotation_id))
        elif annotation_type == 'Gene':
            genes_set.add((annotation_text, annotation_id))

    # Split the passage into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', passage_text)

    filtered_sentences = []
    # Filter sentences that contain both chemicals and genes
    for idx, sentence in enumerate(sentences, start=1):
        contains_chemicals = any(chem in sentence for chem, _ in chemicals_set)
        contains_genes = any(gene in sentence for gene, _ in genes_set)

        if contains_chemicals and contains_genes:
            filtered_sentences.append(sentence)
            print(f"Sentence {idx}: {sentence}")
            for chem, chem_id in chemicals_set:
                if chem in sentence:
                    print(f"Chemical ID: {chem_id}")
            for gene, gene_id in genes_set:
                if gene in sentence:
                    print(f"Gene ID: {gene_id}")
            print()

    #Split filtered sentences by using the word and?

Sentence 9:                 OA-related pathways were found in the KEGG and GO analysis, such as TGF-beta
                signaling pathway, MAPK signaling pathway, hyaluronan metabolic process, and
                chondrocyte differentiation.
Chemical ID: MESH:D006820
Gene ID: 7040

Sentence 5: Studies
                have shown that the MLXIP gene is involved in various biological processes,
                including glucose metabolism, lipid metabolism, and cellular senescence, which have
                also been implicated in the pathogenesis of osteoarthritis.
Chemical ID: MESH:D008055
Chemical ID: MESH:D005947
Gene ID: 22877

Sentence 5: For instance, bisphenol
                A was detected not only in the serum of the OA patients, but also in the synovial
                fluid of knee replacement patients, and exhibited a concentration-dependent
                antagonistic effect on the protective actions of E2 on chondrocyte, which decreased
                the NF-kappaB acti

In [ ]:
# parameter_space = {
#     'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
#     'activation': ['tanh', 'relu'],
#     'solver': ['sgd', 'adam'],
#     'alpha': [0.0001, 0.05],
#     'learning_rate': ['constant','adaptive'],
# }

# from sklearn.model_selection import GridSearchCV

# mlp = MLPClassifier(max_iter=100)

# clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
# clf.fit(X_train, y_train)

In [ ]:
# # Best paramete set
# print('Best parameters found:\n', clf.best_params_)

# # All results
# means = clf.cv_results_['mean_test_score']
# stds = clf.cv_results_['std_test_score']
# for mean, std, params in zip(means, stds, clf.cv_results_['params']):
#     print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
# search map for gene and chemicals
